# Run Example 1

This notbook shows the workflow for running EpiSim37 simulations on your personal computer.

This notebook expects that you have installed pandas and episim37.

In [1]:
import tempfile
from pathlib import Path

import polars as pl
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "jupyterlab"

from episim37.utils import OpenMPSimulator

For running an EpiSim37 simulation we need two things:
1) the EpiSim37 simulation description (the ESL37 file) and
2) the network (nodes and edges)

For this demo we shall use the following files.
The node and edge file can be in parquet or CSV format.

In [2]:
sim_file = "example1.esl37"
node_file = "nodes.parquet"
edge_file = "edges.parquet"

The node table contains the node key (pid for this simulation) and any other static node attributes that the simulation may depend on.

Similarly, the edge table contains the source and target node key (source_pid and target_pid for this simulation) and any other static edge attributes that the simulation may depend on.

The column names in the node and edge file must match those provided in the simulation file.

In [3]:
pl.read_parquet(node_file).head()

pid,is_in_school
i64,bool
3723936,true
3723937,true
3723938,false
3723939,true
3723940,true


In [4]:
pl.read_parquet(edge_file).head()

target_pid,source_pid,duration,is_school_edge,is_non_home_edge
i64,i64,i64,bool,bool
3723936,3723937,14400,false,true
3723936,3723937,20519,false,true
3723936,3723937,41520,false,true
3723936,3731305,1020,false,true
3723936,3742032,1320,false,true


We shall create a working directory for EpiSim37.

EpiSim37 shall use that folder to store the generated C++ codes and the compiled simulator.

We shall also store the input.h5 and output.h5 files in that work directory.

In [5]:
work_dir = Path(Path.home() / "episim37-work-dir")

input_file = work_dir / "input.h5"

In [6]:
simulator = OpenMPSimulator(sim_file, work_dir)

Next we shall call the prepare_build() method.
This uses CMake to find or fetch the C++ libraries needed for building the simulator.
It is not necessary to re-execute this method every time the simulation file is changed.

In [7]:
simulator.prepare_build()

-- Version: 10.1.0
-- Build type: 
-- Configuring done (2.1s)
-- Generating done (0.0s)
-- Build files have been written to: /home/parantapa/episim37-work-dir/build


The build method actually builds the simulator.
It first compules the ESL37 code to C++ and then uses a C++ compiler to build the simulator binary.
It is necessary to rerun this method every time the simulation file is updated.

In [8]:
simulator.build()

[ 50%] Built target fmt
[ 66%] Generating simulator_openmp.cpp, simulator_openmp_common.h
[ 83%] Building CXX object CMakeFiles/simulator_openmp.dir/simulator_openmp.cpp.o
[100%] Linking CXX executable simulator_openmp
[100%] Built target simulator_openmp


Now that the simulator binary has been built, we can add the network to create a full simulation.

The prepare input step creates the input_file from the network files (node and edge file).
This also provides other relevant metadata to the simulator.

In [9]:
simulator.prepare_input(node_file, edge_file, input_file)

Reading node table.

Sorting node table.

Making node index.

### num_nodes:  83313


Reading edge table.

Sorting edge table.

Computing incoming incidence CSR graph's indptr.

### num_edges:  3269342


Creating input file.

Input file created successfully.

We can view the node and edge tables in the input file using extract_nodes() and extract_edges() helper methods.

In [10]:
simulator.extract_nodes(input_file).head()

pid,is_in_school
i64,u8
3723936,1
3723937,1
3723938,0
3723939,1
3723940,1


In [11]:
simulator.extract_edges(input_file).head()

target_pid,source_pid,duration,is_school_edge,is_non_home_edge,_target_node_index,_source_node_index
i64,i64,i64,u8,u8,u32,u32
3723936,3723937,14400,0,1,0,1
3723936,3723937,20519,0,1,0,1
3723936,3723937,41520,0,1,0,1
3723936,3731305,1020,0,1,0,7369
3723936,3733825,5700,1,1,0,9889


Finally to run a simulation, we provide the location of the output file and call simulate.
Here we can also pass a dictionary of configuration variables that were defined in the simulation.

In [12]:
output_file = work_dir / "output.h5"

simulator.simulate(input_file, output_file, num_ticks=100, configs={"enable_hybrid_learning": True}, verbose=True)

### num_threads = 12
### enable_hybrid_learning = 1
### enable_day_30_antigen_test = 1
### enable_day_30_pcr_test = 0
### transmissibility_scale = 0.3
### num_ticks = 100
### input_file = /home/parantapa/episim37-work-dir/input.h5
### output_file = /home/parantapa/episim37-work-dir/output.h5
### num_nodes = 83313
### num_edges = 3269342
### cur_tick = 0
### cur_tick = 1
### cur_tick = 2
### cur_tick = 3
### cur_tick = 4
### cur_tick = 5
### cur_tick = 6
### cur_tick = 7
### cur_tick = 8
### cur_tick = 9
### cur_tick = 10
### cur_tick = 11
### cur_tick = 12
### cur_tick = 13
### cur_tick = 14
### cur_tick = 15
### cur_tick = 16
### cur_tick = 17
### cur_tick = 18
### cur_tick = 19
### cur_tick = 20
### cur_tick = 21
### cur_tick = 22
### cur_tick = 23
### cur_tick = 24
### cur_tick = 25
### cur_tick = 26
### cur_tick = 27
### cur_tick = 28
### cur_tick = 29
### cur_tick = 30
### cur_tick = 31
### cur_tick = 32
### cur_tick = 33
### cur_tick = 34
### cur_tick = 35
### cur_tick = 36
### c

Once the simulation is complete we can extract summary, transitions, transmissions and interventions from the output file.

In [13]:
summary = simulator.extract_summary(output_file)
summary.head()

S,E,Ipresymp,Isymp,Iasymp,R,tick
u64,u64,u64,u64,u64,u64,i64
83308,5,0,0,0,0,0
83303,10,0,0,0,0,1
83298,14,1,0,0,0,2
83293,16,4,0,0,0,3
83285,20,6,1,1,0,4


In [14]:
transitions = simulator.extract_transitions(output_file)
transitions.head()

node_index,state,tick
u32,str,i32
81004,"""Ipresymp""",2
77455,"""Ipresymp""",3
79114,"""Ipresymp""",3
81523,"""Ipresymp""",3
4885,"""E""",4


In [15]:
transmissions = simulator.extract_transmissions(output_file)
transmissions.head()

edge_index,state,tick
u64,str,i32
202243,"""E""",4
2492689,"""E""",4
3231558,"""E""",4
1439364,"""E""",5
3136826,"""E""",5


In [16]:
interventions = simulator.extract_interventions(output_file)
interventions.head()

node_index,state,tick
u32,str,i32
78292,"""E""",0
78538,"""E""",0
79562,"""E""",0
81004,"""E""",0
82092,"""E""",0


Runtime statistics are also saved in the output file

In [17]:
simulator.runtime_stats(output_file)

{'mem_use_gb': 0.076155456,
 'init_time_s': 0.026321744997403584,
 'main_time_s': 0.6150129739980912}

Configuration variables are also saved in output file

In [18]:
simulator.extract_configs(output_file)

{'enable_hybrid_learning': 1,
 'enable_day_30_antigen_test': 1,
 'enable_day_30_pcr_test': 0,
 'transmissibility_scale': 0.30000001192092896}

Compute attack rate as follows:

In [19]:
num_nodes = simulator.num_nodes(input_file)
attack_rate = simulator.compute_cum_count(transitions, ["E"]) / num_nodes
attack_rate

0.8127063003372823

In [20]:
states = summary.columns
states.remove("tick")

title = "Number of individuals in each state over time"
pdf = summary.melt(id_vars="tick", value_vars=states, variable_name="state", value_name="count")
px.line(pdf, x="tick", y="count", color="state", title=title)

In [21]:
num_ticks = simulator.num_ticks(output_file)
state_tick_cum_counts = simulator.compute_state_tick_cum_counts(transitions, num_ticks)

states = state_tick_cum_counts.columns
states.remove("tick")

title = "Cumulative number of individuals who have visited each state over time"
pdf = state_tick_cum_counts.melt(id_vars="tick", value_vars=states, variable_name="state", value_name="count")
px.line(pdf, x="tick", y="count", color="state", title=title)